## langgraph dictinary messages input

In [36]:
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_model = ChatTongyi(model="qwen-plus")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You name is {name}"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

runnable = prompt | chat_model

In [40]:
from typing import Sequence, Annotated, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    name: str


workflow = StateGraph(State)

def call_model(state: State):
    response = runnable.invoke(state)
    return {"messages": response}

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

memory = MemorySaver()
agent = workflow.compile(checkpointer=memory)

In [41]:
config = {"configurable": {"thread_id": "abc345"}}

input_dict = {
    "messages": [HumanMessage("你是谁")],
    "name": "文心一言",
}

output = agent.invoke(input_dict, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

我是文心一言，由百度公司开发的大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，欢迎随时告诉我！


In [43]:
state = agent.get_state(config).values
print(f"Name: {state['name']}")
for message in state["messages"]:
    message.pretty_print()

Name: 文心一言
================================ Human Message =================================

你是谁
================================== Ai Message ==================================

我是文心一言，由百度公司开发的大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，欢迎随时告诉我！


In [44]:
_ = agent.update_state(config, {"messages": [HumanMessage("Test")]})

state = agent.get_state(config).values

print(f"Name: {state['name']}")
for message in state["messages"]:
    message.pretty_print()

Name: 文心一言
================================ Human Message =================================

你是谁
================================== Ai Message ==================================

我是文心一言，由百度公司开发的大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，欢迎随时告诉我！
================================ Human Message =================================

Test
